# HPC- Project Stage 1
## Pranav Rao- 15008121

In [1]:
import numpy as np
import pyopencl as cl
from scipy.sparse.linalg import LinearOperator, gmres, bicgstab
import matplotlib.pyplot as plt
ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx)

In [2]:
# Normally Distributed sigma field
def make_random_sigma(N, mean=2, sd=0.5, seed=0):
    S = np.random.RandomState(seed).normal(mean, sd, (N, N))
    sigma = np.exp(np.negative(S))
    return sigma

Fixed $\sigma$ used to compare the results of the OpenCL kernel with FEniCs:

$$\sigma (x,y) = 1 + x^2 + y^2$$

In [3]:
# Sigma field defined as a specific function of x and y for comparison with FEniCs
def make_sigma(N):
    points = np.linspace(0,1,N)
    
    kernel ="""
    __kernel void make_sigma(__global double *points,
                             __global double *sigma)
    {
        int N = get_global_size(0);
        
        int i = get_global_id(0);
        int j = get_global_id(1);

        sigma[i * N + j] = 1 + points[i]*points[i] + points[j]*points[j];
    }
    """
    
    prg = cl.Program(ctx, kernel)
    prg.build()
    make_sigma_kernel = prg.make_sigma
    mf = cl.mem_flags        
    
    points_buffer = cl.Buffer(ctx, mf.COPY_HOST_PTR | mf.READ_ONLY, hostbuf=points)
    sigma_buffer = cl.Buffer(ctx, mf.ALLOC_HOST_PTR, size = N * N * 8)
    
    make_sigma_kernel(queue, (N, N), (1, 1), points_buffer, sigma_buffer)
    
    sigma, _ = cl.enqueue_map_buffer(queue, sigma_buffer, cl.map_flags.READ, 0, (N*N,), np.double)
    
    queue.finish()
    
    return sigma

Poisson Equation that we are interested in solving using the finite difference method:

$$-\nabla \cdot \left(\sigma(x, y)\nabla\right)u(x, y) = f(x, y)$$ 

In [4]:
def solve_poisson_equation(u):
    """Matrix free opencl implementation of finite difference methods
       to solve the given poisson equation.
       Uses the 5 point stencil to evaluate the sccuessive value of u.
    """
    
    kernel = """
        __kernel void solve_poisson_equation(__global double *u,
                                             __global double *sigma,
                                             __global double *result)
        {
            int N = get_global_size(0);
            double h = (double) 1.0/(N+1);
            
            int i = get_global_id(0);
            int j = get_global_id(1);
            
            // 5 Point Stencil for u and sigma
            
            double u_up; double u_down; double u_right; double u_left; double u_centre;
            double sigma_up; double sigma_down; double sigma_right; double sigma_left; double sigma_centre;
            
            if (i == 0) {
                u_up = 0;
                sigma_up = sigma[(i + 1) * N + j];
            } else {
                u_up = u[(i - 1) * N + j];
                sigma_up = sigma[(i - 1) * N + j];
            }
            
            if(j == 0) {
                u_left = 0;
                sigma_left = sigma[i * N + j + 1];
            }else {
                u_left = u[i * N + j - 1];
                sigma_left = sigma[i * N + j - 1];
            }
            
            if(j == (N - 1)) {
                u_right = 0;
                sigma_right = sigma[i * N + j - 1];
            }else {
                u_right = u[i * N + j + 1];
                sigma_right = sigma[i * N + j + 1];
            }
            
            if(i == (N - 1)){
                u_down = 0;
                sigma_down = sigma[(i - 1) * N + j];
            }else {
                u_down = u[(i + 1) * N + j];
                sigma_down = sigma[(i + 1) * N + j];
            }
                
            u_centre = u[i * N + j];
            sigma_centre = sigma[i * N + j];
            
            result[i * N + j] = -1.0 * ((0.5 * (sigma_down + sigma_centre) * (u_down - u_centre)/h -
                                0.5 * (sigma_up + sigma_centre) * (u_centre - u_up)/h)/h +
                                (0.5 * (sigma_right + sigma_centre) * (u_right - u_centre)/h -
                                0.5 * (sigma_left + sigma_centre) * (u_centre - u_left)/h)/h);
            
        }

        """

    prg = cl.Program(ctx, kernel)
    prg.build()
    poisson_equation_kernel = prg.solve_poisson_equation
    mf = cl.mem_flags
    
    sigma_buffer = cl.Buffer(ctx, mf.COPY_HOST_PTR | mf.READ_ONLY, hostbuf=sigma)
    u_buffer = cl.Buffer(ctx, mf.COPY_HOST_PTR | mf.READ_WRITE, hostbuf=u)
    result_buffer = cl.Buffer(ctx, mf.ALLOC_HOST_PTR, size = N * N * 8)

    poisson_equation_kernel(queue, (N, N), (1, 1), u_buffer, sigma_buffer, result_buffer)
    
    result, _ = cl.enqueue_map_buffer(queue, result_buffer, cl.map_flags.READ, 0, (N*N,), np.double)

    queue.finish()

    return result


In [10]:
N = 10
# sigma = make_sigma(N)
sigma = make_random_sigma(N)

A = LinearOperator((N*N,N*N),matvec=solve_poisson_equation)
f = np.float64(np.ones(N*N))

In [11]:
residuals_gmres = []
def res_gmres(rk):
    res = np.linalg.norm(rk)
    residuals_gmres.append(res)
    
u_gmres, info_gmres = gmres(A, f, callback = res_gmres)
residuals_gmres = np.array(residuals_gmres)
u_gmres_mat = u_gmres.reshape(N,N)

In [13]:
solve_poisson_equation(u_gmres)

array([0.99999917, 0.99999224, 1.00000615, 1.00001005, 1.00000063,
       1.00000482, 0.99999923, 1.00000354, 1.00000167, 0.99999731,
       0.99999213, 0.9999859 , 1.00000241, 1.00000936, 1.00001078,
       0.99999844, 0.99999678, 0.99999782, 1.00000211, 1.00000237,
       0.99999349, 0.99999299, 0.99999394, 1.00000001, 1.00000339,
       0.99999338, 0.99999336, 0.99998605, 0.99999263, 0.99999995,
       0.99999286, 0.99999276, 0.9999957 , 0.99998724, 0.99999184,
       0.99998665, 0.99998708, 0.99999259, 0.99999188, 1.00000338,
       0.99998911, 0.99999575, 1.00000229, 1.00000178, 0.99998598,
       0.99999236, 0.99998118, 0.99999814, 0.99998962, 1.0000054 ,
       1.00000025, 0.99999847, 0.99999815, 1.0000126 , 1.00000052,
       0.99999459, 0.99999277, 0.99999607, 1.00000672, 0.99999634,
       1.00000213, 1.00000341, 1.00001224, 1.00000106, 1.0000153 ,
       0.99999974, 1.00000045, 1.00000821, 0.99999949, 1.00000277,
       1.00000338, 1.00000078, 1.00000059, 1.00001397, 1.00001

In [ ]:
def add_boundary(u_mat):
    N = len(u_mat)+2
    u = np.zeros((N,N))
    for i in range(1,N-1):
        for j in range(1,N-1):
            u[i][j] = u_mat[i-1][j-1]
    return u

In [ ]:
u_gmres = add_boundary(u_gmres_mat)

plt.figure(dpi=200,figsize=(5,5))
plt.imshow(u_gmres,extent=[0,1,0,1],origin="lower")
plt.xlabel("x")
plt.ylabel("y")
plt.title("GMRes solution, N = {}".format(N))

print(u_gmres[1,1])
print(u_gmres[10,10])
print(u_gmres[22,16])
print(u_gmres[15,15])

In [ ]:
residuals_bicgstab = []
def res_bicgstab(xk):
    res = np.linalg.norm(f - A * xk)
    residuals_bicgstab.append(res/np.linalg.norm(f))
    
u_bicgstab , info_bicgstab = bicgstab(A,f, callback = res_bicgstab)
residuals_bicgstab = np.array(residuals_bicgstab)
u_bicgstab_mat = u_bicgstab.reshape(N,N)

In [ ]:
u_bicgstab = add_boundary(u_bicgstab_mat)

plt.figure(2,dpi=200,figsize=(5,5))
plt.imshow(u_bicgstab,extent=[0,1,0,1],origin="lower")
plt.xlabel("x")
plt.ylabel("y")
plt.title("BiCGSTAB Solution, N = {}".format(N))

print(u_bicgstab[1,1])
print(u_bicgstab[10,10])
print(u_bicgstab[22,16])
print(u_bicgstab[15,15])

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

x_gmres = np.arange(1,len(residuals_gmres)+1)
x_bicgstab = np.arange(1,len(residuals_bicgstab)+1)

plt.figure(2,dpi=100,figsize=(8,5))
plt.semilogy(x_gmres, residuals_gmres, "b-", label="gmres")
plt.semilogy(x_bicgstab, residuals_bicgstab, "r-", label="bicgstab")
plt.ylabel("Norm of Residuals")
plt.xlabel("Number of Iterations")
plt.legend()
plt.grid(linestyle="--")

plt.show()

In [ ]:
iterations_gmres = []
iterations_bicgstab = []
residuals_gmres = []
residuals_bicgstab = []
Npoints = []

for N in range(10,55,5):
    Npoints.append(N)
    residuals_gmres = []
    residuals_bicgstab = []
    sigma = make_sigma(N)
    A = LinearOperator((N*N,N*N),matvec=solve_poisson_equation,)
    f = np.float64(np.ones(N*N))
    u_gmres, info_gmres = gmres(A, f, callback = res_gmres)
    u_bicgstab , info_bicgstab = bicgstab(A,f, callback = res_bicgstab)
    iterations_gmres.append(len(residuals_gmres))
    iterations_bicgstab.append(len(residuals_bicgstab))
                               
iterations_gmres = np.array(iterations_gmres)
iterations_bicgstab = np.array(iterations_bicgstab)
Npoints = np.array(Npoints)                               

In [ ]:
from scipy.stats import linregress

xpoints = np.linspace(9,101)
slope, intercept, rvalue, pvalue, stderr = linregress(np.log10(Npoints),np.log10(iterations_gmres))
print("gmres fit: y = {} x + {} with rvalue: {}".format(slope,intercept,rvalue))
y_gmres = slope*np.log10(xpoints) + intercept
y_gmres = 10**y_gmres

In [ ]:
slope, intercept, rvalue, pvalue, stderr = linregress(np.log10(Npoints),np.log10(iterations_bicgstab))
print("bicgstab fit: y = {} x + {} with rvalue: {}".format(slope,intercept,rvalue))
y_bicgstab = slope*np.log10(xpoints) + intercept
y_bicgstab = 10**y_bicgstab

In [ ]:
plt.figure(dpi=200,figsize=(8,5))
plt.xlabel("Number of discretisation points")
plt.ylabel("Number of iterations for convergence")

plt.semilogy((xpoints),(y_gmres), "g-",label = "gmres fit")
plt.semilogy((Npoints),(iterations_gmres),"rx",label="gmres")
plt.semilogy((xpoints),(y_bicgstab), "k-", label = "bicgstab fit")
plt.semilogy((Npoints),(iterations_bicgstab),"bx",label="bicgstab")
plt.grid(linestyle="--")
plt.legend()